In [1]:
%matplotlib inline

In [2]:
from thetis import *
from firedrake_adjoint import *

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from adapt_utils.unsteady.solver_adjoint import AdaptiveDiscreteAdjointProblem
from adapt_utils.case_studies.tohoku.options.box_options import TohokuBoxBasisOptions
from adapt_utils.case_studies.tohoku.options.radial_options import TohokuRadialBasisOptions

### Taylor test tsunami propagation

First, solve forward for a short amount of time and check that the we can accurately compute the gradient of the norm of the final solution tuple w.r.t. the initial solution tuple.

In [5]:
kwargs = {
    'level': 0,
    'end_time': 60,
    'debug': True,
    'debug_mode': 'light',
}
nonlinear = False

In [6]:
kwargs_src = kwargs.copy()
gaussian_scaling = 6.0
kwargs_src['control_parameters'] = [gaussian_scaling, ]
kwargs_src['nx'], kwargs_src['ny'] = 1, 1
op_src = TohokuRadialBasisOptions(**kwargs_src)
swp = AdaptiveDiscreteAdjointProblem(op_src, nonlinear=nonlinear)
swp.set_initial_condition()
f_src = swp.fwd_solutions[0].split()[1]

INIT: Loading mesh...
INIT: Checking validity of gauge coordinates...
INIT: QoI normalisation factor (in time): 2.5253e-05
INIT: overall QoI normalisation factor: 2.5253e-05
INIT: Assigning control parameter values...
********************************************************************************
AdaptiveDiscreteAdjointProblem initialisation begin
********************************************************************************

SETUP: Setting default meshes...
SETUP: Mesh 0 has 15849 elements and 8322 vertices
SETUP: Creating finite elements...
SETUP: Creating function spaces...
SETUP: Creating solutions...
SETUP: Creating fields...
INIT: Reading bathymetry file...
INIT: Creating bathymetry interpolator...
INIT: Interpolating bathymetry...
INIT: Capping bathymetry...
INIT: Computing CFL number on mesh 0...
INIT:   dx = 3.1817e+03  dt = 6.0000e+01  CFL number = 5.8199e+00 > 1
SETUP: Setting stabilisation parameters...
SETUP: Setting boundary conditions...
SETUP: Creating output files..

Create another `Options` object on the same mesh which uses a basis array.

In [7]:
op = TohokuBoxBasisOptions(mesh=op_src.default_mesh, **kwargs)
# op = TohokuRadialBasisOptions(mesh=op_src.default_mesh, **kwargs)

INIT: Loading mesh...
INIT: Checking validity of gauge coordinates...
INIT: QoI normalisation factor (in time): 2.5253e-05
INIT: overall QoI normalisation factor: 2.5253e-05
INIT: Assigning control parameter values...


Project the source field into the array basis.

In [8]:
swp = AdaptiveDiscreteAdjointProblem(op, nonlinear=nonlinear)
op.project(swp, f_src)
kwargs['control_parameters'] = [m.dat.data[0] for m in op.control_parameters]

********************************************************************************
AdaptiveDiscreteAdjointProblem initialisation begin
********************************************************************************

SETUP: Setting default meshes...
SETUP: Mesh 0 has 15849 elements and 8322 vertices
SETUP: Creating finite elements...
SETUP: Creating function spaces...
SETUP: Creating solutions...
SETUP: Creating fields...
INIT: Reading bathymetry file...
INIT: Creating bathymetry interpolator...
INIT: Interpolating bathymetry...
INIT: Capping bathymetry...
INIT: Computing CFL number on mesh 0...
INIT:   dx = 3.1817e+03  dt = 6.0000e+01  CFL number = 5.8199e+00 > 1
SETUP: Setting stabilisation parameters...
SETUP: Setting boundary conditions...
SETUP: Creating output files...
INIT: Assembling rotated array of indicator functions...


In [9]:
# tricontourf(op.basis_functions[62].split()[1]);

In [10]:
swp.clear_tape()

In [11]:
swp.set_initial_condition()

INIT: Assembling initial surface...
INIT: Subtracting initial surface from bathymetry field...
INIT: Interpolating bathymetry...
INIT: Capping bathymetry...


In [13]:
init = swp.fwd_solutions[0].copy(deepcopy=True)
control = Control(swp.fwd_solutions[0])

In [14]:
swp.setup_solver_forward_step(0)
swp.solve_forward_step(0)

SETUP: Creating forward equations on mesh 0...
SETUP: Creating forward timesteppers on mesh 0...
SETUP: Adding callbacks on mesh 0...
SETUP: Creating CallbackManagers...
SOLVE: Entering forward timeloop on mesh 0...
FORWARD SOLVE  time     0.00  (  0.00) seconds
INIT: Applying UTide de-tiding algorithm to gauge 801...
INIT: Applying UTide de-tiding algorithm to gauge 802...
INIT: Applying UTide de-tiding algorithm to gauge 803...
INIT: Applying UTide de-tiding algorithm to gauge 804...
INIT: Applying UTide de-tiding algorithm to gauge 806...
INIT: Applying UTide de-tiding algorithm to gauge 807...
INIT: Applying UTide de-tiding algorithm to gauge P02...
INIT: Applying UTide de-tiding algorithm to gauge P06...
INIT: Applying UTide de-tiding algorithm to gauge KPG1...
INIT: Applying UTide de-tiding algorithm to gauge KPG2...
INIT: Applying UTide de-tiding algorithm to gauge 21418...
    elevation norm           at time    0.0 =  4.6759e+05
    velocity norm            at time    0.0 =  0

In [21]:
# u, eta = init.split()
# fig, axes = plt.subplots(figsize=(8, 7))
# tricontourf(eta, axes=axes, cmap='coolwarm', levels=50)
# axes.axis(False);

# u, eta = swp.fwd_solutions[0].split()
# fig, axes = plt.subplots(figsize=(8, 7))
# tricontourf(eta, axes=axes, cmap='coolwarm', levels=50)
# axes.axis(False);

QoI: $J=\int_\Omega\mathbf q(\mathbf x,T)\cdot\mathbf q(\mathbf x,T)\;\mathrm dx$

Control: $\mathbf c=\mathbf q(\mathbf x,0)$

In [15]:
q = swp.fwd_solutions[0]
J = assemble(inner(q, q)*dx)
Jhat = ReducedFunctional(J, control)

Option 1: test the gradient in a random direction for a random initial condition

In [16]:
# c = Function(q)
# c.dat.data[0][:] = np.random.rand(*c.dat.data[0].shape)
# c.dat.data[1][:] = np.random.rand(*c.dat.data[1].shape)

Option 2: test the gradient in a random direction at the chosen initial condition

In [17]:
c = Function(init)

In [18]:
dc = Function(q)
dc.dat.data[0][:] = np.random.rand(*dc.dat.data[0].shape)
dc.dat.data[1][:] = np.random.rand(*dc.dat.data[1].shape)

In [19]:
minconv = taylor_test(Jhat, c, dc)
assert minconv > 1.90

Running Taylor test
Computed residuals: [3436683586.907537, 859196252.6811701, 214811672.761926, 53711484.24494616]
Computed convergence rates: [1.9999574236396214, 1.9999153103318845, 1.9997698966983544]


### Timeseries QoI

Everything seems to be okay for the simple norm functional. What about the timeseries QoI (that we actually care about)?

QoI: $J=\text{[timeseries misfit]}$

Control: $\mathbf c=\mathbf q(\mathbf x,0)$

In [24]:
Jhat = ReducedFunctional(op.J, control)

In [25]:
minconv = taylor_test(Jhat, c, dc)
assert minconv > 1.90

Running Taylor test
Computed residuals: [7.272883430422685e-07, 1.8185597708857077e-07, 4.549420260457722e-08, 1.1393509485495259e-08]
Computed convergence rates: [1.9997311091540773, 1.9990417265404195, 1.997470510593494]


In [26]:
swp.clear_tape()

### Projection

The forward propagation is okay. What about the projection from box basis to $\mathbb P2$?

In [27]:
# control_values = np.random.rand(len(op.control_parameters))
control_values = [m.dat.data[0] for m in op.control_parameters]
op.assign_control_parameters(control_values)
controls = [Control(m) for m in op.control_parameters]

INIT: Assigning control parameter values...


In [28]:
swp.set_initial_condition()

INIT: Assembling initial surface...
INIT: Subtracting initial surface from bathymetry field...
INIT: Interpolating bathymetry...
INIT: Capping bathymetry...


QoI: $J=\int_\Omega\mathbf q(\mathbf x,T)\cdot\mathbf q(\mathbf x,T)\;\mathrm dx$

Control: $\mathbf c=\text{[basis coefficients for source]}$

In [29]:
q = swp.fwd_solutions[0]
J = assemble(inner(q, q)*dx)
Jhat = ReducedFunctional(J, controls)

Option 1: test the gradient in a random direction for a random initial condition

In [30]:
# c = [Function(m) for m in op.control_parameters]
# for m in c:
#     m.dat.data[0] = np.random.random()

Option 2: test the gradient in a random direction at the chosen initial condition

In [33]:
c = [Function(m) for m in op.control_parameters]

In [31]:
dc = [Function(m) for m in op.control_parameters]
for m in dc:
    m.dat.data[0] = np.random.random()

In [34]:
minconv = taylor_test(Jhat, c, dc)
assert minconv > 1.90

Running Taylor test
Computed residuals: [3778616.9859530926, 944654.246470809, 236163.5612847209, 59040.89032253623]
Computed convergence rates: [2.0000000000266716, 2.0000000020341435, 1.9999999999668654]


In [35]:
swp.clear_tape()

### Combined

The projection is also okay. Let's check everything is okay when the two are chained together.

In [36]:
op.assign_control_parameters(control_values)
controls = [Control(m) for m in op.control_parameters]

INIT: Assigning control parameter values...


In [37]:
swp.set_initial_condition()
init = swp.fwd_solutions[0].copy(deepcopy=True)

INIT: Assembling initial surface...
INIT: Subtracting initial surface from bathymetry field...
INIT: Interpolating bathymetry...
INIT: Capping bathymetry...


In [38]:
swp.setup_solver_forward_step(0)
swp.solve_forward_step(0)

SETUP: Creating forward equations on mesh 0...
SETUP: Creating forward timesteppers on mesh 0...
SETUP: Adding callbacks on mesh 0...
SETUP: Creating CallbackManagers...
SOLVE: Entering forward timeloop on mesh 0...
FORWARD SOLVE  time     0.00  (  0.00) seconds
INIT: Applying UTide de-tiding algorithm to gauge 801...
INIT: Applying UTide de-tiding algorithm to gauge 802...
INIT: Applying UTide de-tiding algorithm to gauge 803...
INIT: Applying UTide de-tiding algorithm to gauge 804...
INIT: Applying UTide de-tiding algorithm to gauge 806...
INIT: Applying UTide de-tiding algorithm to gauge 807...
INIT: Applying UTide de-tiding algorithm to gauge P02...
INIT: Applying UTide de-tiding algorithm to gauge P06...
INIT: Applying UTide de-tiding algorithm to gauge KPG1...
INIT: Applying UTide de-tiding algorithm to gauge KPG2...
INIT: Applying UTide de-tiding algorithm to gauge 21418...
    elevation norm           at time    0.0 =  4.6759e+05
    velocity norm            at time    0.0 =  0

QoI: $J=\int_\Omega\mathbf q(\mathbf x,T)\cdot\mathbf q(\mathbf x,T)\;\mathrm dx$

Control: $\mathbf c=\text{[basis coefficients for source]}$

In [39]:
q = swp.fwd_solutions[0]
J = assemble(inner(q, q)*dx)
Jhat = ReducedFunctional(J, controls)

In [40]:
minconv = taylor_test(Jhat, c, dc)
assert minconv > 1.90

Running Taylor test
Computed residuals: [3356300.35702312, 839132.0311726928, 209811.4297861755, 52467.06232045591]
Computed convergence rates: [1.9999020981138973, 1.9998045528503388, 1.9996093534703405]


### Timeseries QoI

Everything seems to be okay for the simple norm functional. What about the timeseries QoI (that we actually care about)?

QoI: $J=\text{[timeseries misfit]}$

Control: $\mathbf c=\text{[basis coefficients for source]}$

In [41]:
Jhat = ReducedFunctional(op.J, controls)

In [42]:
minconv = taylor_test(Jhat, c, dc)
assert minconv > 1.90

Running Taylor test
Computed residuals: [7.513347372461798e-11, 1.8866812251134415e-11, 4.7583224411293645e-12, 1.2103767096586157e-12]
Computed convergence rates: [1.9936051206021486, 1.9873257424173378, 1.974996905035385]


All good!